In [9]:
from datetime import datetime, timedelta
import requests

today = datetime.today()

for i in range(2):
    targetDt = (today + timedelta(days=-(i+1)*7)).strftime('%Y%m%d')
    # print(targetDt)

    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json'
    key = '660f73acbf0225280f5db341b9f4e840'
    movie_url = f'{url}?key={key}&targetDt={targetDt}'

    res = requests.get(movie_url).json()
    print(res)

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스', 'showRange': '20191115~20191115', 'dailyBoxOfficeList': [{'rnum': '1', 'rank': '1', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20195002', 'movieNm': '블랙머니', 'openDt': '2019-11-13', 'salesAmt': '1400310100', 'salesShare': '41.6', 'salesInten': '428855480', 'salesChange': '44.1', 'salesAcc': '3567413030', 'audiCnt': '160475', 'audiInten': '37886', 'audiChange': '30.9', 'audiAcc': '429677', 'scrnCnt': '1053', 'showCnt': '4926'}, {'rnum': '2', 'rank': '2', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20181434', 'movieNm': '신의 한 수: 귀수편', 'openDt': '2019-11-07', 'salesAmt': '714595680', 'salesShare': '21.2', 'salesInten': '109768930', 'salesChange': '18.1', 'salesAcc': '13338027600', 'audiCnt': '81750', 'audiInten': '5859', 'audiChange': '7.7', 'audiAcc': '1555571', 'scrnCnt': '988', 'showCnt': '4438'}, {'rnum': '3', 'rank': '3', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20191029', 'movieNm': '82년생 김지영', 'openD

In [14]:
movieNm_list = []
for i in range(10):
    movieNm = res.get('boxOfficeResult').get('dailyBoxOfficeList')[i].get('movieNm')
    if movieNm not in movieNm_list:
        movieNm_list.append(movieNm)
print(movieNm_list)

['블랙머니', '신의 한 수: 귀수편', '82년생 김지영', '터미네이터: 다크 페이트', '엔젤 해즈 폴른', '좀비랜드: 더블 탭', '날씨의 아이', '윤희에게', '아담스 패밀리', '조커']


In [28]:
import urllib.request
import json
from bs4 import BeautifulSoup
 
#애플리케이션 클라이언트 id 및 secret
client_id = "hG6O_G7PA_2DCtQbUri0" 
client_secret = "fsSh41QFYc"


#영화검색 url
url = "https://openapi.naver.com/v1/search/movie.json"
option = "&display=1"

for movie_name in movieNm_list:

    query = "?query="+urllib.parse.quote(movie_name) # movieNm
    url_query = url + query + option

    #Open API 검색 요청 개체 설정
    request = urllib.request.Request(url_query)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    #검색 요청 및 처리
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode == 200):
        response_body = response.read()
        str_info = response_body.decode('utf-8')
        # print(type(temp))
        dict_info = json.loads(str_info)
#         print(dict_info.get('items')[0].get('title').replace('<b>', '').replace('</b>', ''))
#         print(dict_info.get('items')[0].get('link')) # 줄거리 크롤링에 필요
#         print(dict_info.get('items')[0].get('director'))

#         print(dict_info.get('items')[0].get('subtitle').replace('<b>', '').replace('</b>', ''))
#         print(dict_info.get('items')[0].get('userRating')) # string type
#         print(dict_info.get('items')[0].get('image'))
#         print()
        link = dict_info.get('items')[0].get('link')
        
        response = requests.get(link).text
        soup = BeautifulSoup(response, 'html.parser')


        contents = soup.select('div.story_area > p.con_tx')
        # print(contents)

        # 줄거리
        movie_info = []
        if len(contents) == 0:
            movie_info.append("줄거리 없음")
        else:
            for c in contents:
                temp = c.text
                temp = temp.replace('\r', '')
                temp = temp.replace('\xa0', ' ')
                movie_info.append(temp)
        print(movie_info)
    else:
        print("Error code:"+rescode)

['일명 서울지검 ‘막프로’! 검찰 내에서 거침없이 막 나가는 문제적 검사로  이름을 날리는 ‘양민혁’은 자신이 조사를 담당한 피의자가 자살하는 사건으로 인해  하루 아침에 벼랑 끝에 내몰린다. 억울한 누명을 벗기 위해 내막을 파헤치던 그는  피의자가 대한은행 헐값 매각사건의 중요 증인이었음을 알게 된다.   근거는 의문의 팩스 5장! 자산가치 70조 은행이 1조 7천억원에 넘어간  희대의 사건 앞에서 ‘양민혁’ 검사는 금융감독원, 대형 로펌, 해외펀드 회사가 뒤얽힌  거대한 금융 비리의 실체와 마주하게 되는데…   대한민국 최대의 금융스캔들, 사건은 아직 끝나지 않았다!']
['바둑으로 모든 것을 잃은 아이 ‘귀수’ 유일하게 기대던 스승 허일도마저 잃고 홀로 살아남아 세상을 향한 복수를 계획한다. 운명의 선택은 神의 놀음판에 있다! 자신을 사지로 내몬 냉혹한 내기바둑판으로 뛰어든 귀수(권상우)는 전국을 돌아다니며 귀신 같이 바둑을 두는 자들과 대결을 펼치는데…  사활을 건 대결! 신의 한 수를 다시 두시겠습니까?']
['1982년 봄에 태어나 누군가의 딸이자 아내, 동료이자 엄마로 2019년 오늘을 살아가는 ‘지영’(정유미). 때론 어딘가 갇힌 듯 답답하기도 하지만 남편 ‘대현’(공유)과 사랑스러운 딸, 그리고 자주 만나지 못해도 항상 든든한 가족들이 ‘지영’에겐 큰 힘이다.  하지만 언젠가부터 마치 다른 사람이 된 것처럼 말하는 ‘지영’. ‘대현’은 아내가 상처 입을까 두려워 그 사실을 털어놓지 못하고 ‘지영’은 이런 ‘대현’에게 언제나 “괜찮다”라며 웃어 보이기만 하는데…  모두가 알지만 아무도 몰랐던 당신과 나의 이야기']
['심판의 날 그 후, 뒤바뀐 미래  새로운 인류의 희망 ‘대니’(나탈리아 레이즈)를 지키기 위해 슈퍼 솔져 ‘그레이스’(맥켄지 데이비스)가 미래에서 찾아오고, ‘대니’를 제거하기 위한 터미네이터 ‘Rev-9’(가브리엘 루나)의 추격이 시작된다.   최첨단 기술력으로 무장한 최강의 적 터미네이터 ‘Rev-9’의 무차별적인 공격에 쫓기